<a href="https://colab.research.google.com/github/Kethanvr/Reag-VecotrDB/blob/main/Final_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Install Required Packages
!pip install -q pymongo sentence-transformers google-generativeai langchain langchain-google-genai langchain-community pypdf python-docx openpyxl pandas unstructured pillow langchain-text-splitters
!pip install -q duckduckgo-search beautifulsoup4 requests
!pip install -q rich

print("✅ Dependencies installed successfully.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.1/719.1 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
   ━━━━

In [7]:
pip install duckduckgo-search

In [2]:
# @title 2. Load API Keys & Imports
import os
import hashlib
import numpy as np
import uuid
import json
import requests
from typing import List, Dict, Any, Optional
from io import BytesIO

# ML & AI Imports
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from pymongo import MongoClient
from google.colab import userdata, files

# LangChain Imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    PyPDFLoader, Docx2txtLoader, TextLoader,
    UnstructuredExcelLoader, CSVLoader,
    UnstructuredMarkdownLoader, UnstructuredHTMLLoader
)
from langchain_core.documents import Document

# Web & Search Imports
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from PIL import Image

# UI Imports
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.table import Table
from rich import box

# Configuration
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    MONGODB_URI = userdata.get('MONGODB_URI')
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ API Keys loaded successfully!")
except Exception as e:
    print(f"❌ Error loading keys: {e}")
    print("Please set GEMINI_API_KEY and MONGODB_URI in Colab Secrets.")

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


✅ API Keys loaded successfully!


In [16]:
# @title 3. Define Advanced RAG Class (Fixed Web Search)
import os
import hashlib
import uuid
import time
import requests
import warnings
from typing import List, Dict, Any
from io import BytesIO
from PIL import Image
from bs4 import BeautifulSoup

# Try to import DDGS, handling potential version issues
try:
    from duckduckgo_search import DDGS
except ImportError:
    # Fallback or re-install hint
    print("⚠️ Please run: !pip install -U duckduckgo-search")

# ML & Database Imports
import google.generativeai as genai
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
from google.colab import userdata, files

# LangChain Imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    PyPDFLoader, Docx2txtLoader, TextLoader, CSVLoader,
    UnstructuredHTMLLoader
)

class AdvancedRAGWithMemoryVisionWeb:
    def __init__(self, mongodb_uri: str, db_name: str, collection_name: str, model_name: str):
        print("🔄 Loading Embedding model (BAAI/bge-large-en-v1.5)...")
        self.embedding_model = SentenceTransformer('BAAI/bge-large-en-v1.5')

        print(f"🔄 Connecting to MongoDB (DB: {db_name})...")
        self.client = MongoClient(mongodb_uri)
        self.db = self.client[db_name]
        self.collection = self.db[collection_name]
        self.memory_collection = self.db["conversation_memory"]

        self.model_name = model_name
        print(f"🔄 Initializing {self.model_name}...")
        self.llm = genai.GenerativeModel(self.model_name)

        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len)
        self.session_id = str(uuid.uuid4())
        self.conversation_history = []
        print("✅ System Initialized!")

    # --- MEMORY ---
    def add_to_memory(self, role: str, content: str):
        msg = {"role": role, "content": content, "timestamp": str(uuid.uuid4())}
        self.conversation_history.append(msg)
        if self.session_id:
            self.memory_collection.insert_one({**msg, "session_id": self.session_id})

    def get_context_str(self, last_n=5):
        return "\n".join([f"{m['role']}: {m['content']}" for m in self.conversation_history[-last_n:]])

    # --- WEB SEARCH (FIXED) ---
    def web_search_and_scrape(self, query: str, max_results=3):
        print(f"🌐 Searching DuckDuckGo for: '{query}'...")
        try:
            ddgs = DDGS()
            # Use the .text() method which is standard for the new version
            results = list(ddgs.text(query, max_results=max_results))

            if not results:
                return ""

            context = []
            for r in results:
                context.append(f"Source: {r.get('title')}\nURL: {r.get('href')}\nSummary: {r.get('body')}")

            return "\n\n".join(context)
        except Exception as e:
            print(f"⚠️ Web Search Error: {e}")
            return ""

    # --- VISION ---
    def analyze_uploaded_image(self, uploaded_file_dict, question="Describe this image"):
        try:
            filename = list(uploaded_file_dict.keys())[0]
            image = Image.open(BytesIO(uploaded_file_dict[filename]))
            response = self.llm.generate_content([question, image])
            return response.text
        except Exception as e:
            return f"Error processing image: {str(e)}"

    # --- INGESTION ---
    def ingest_file(self, file_path: str, metadata: Dict = None):
        ext = os.path.splitext(file_path)[1].lower()
        try:
            if ext == '.pdf': loader = PyPDFLoader(file_path)
            elif ext in ['.docx', '.doc']: loader = Docx2txtLoader(file_path)
            elif ext == '.csv': loader = CSVLoader(file_path)
            else: loader = TextLoader(file_path)

            docs = loader.load()
            if not docs: return 0

            chunks = self.text_splitter.split_documents(docs)
            new_chunks = 0

            for i, chunk in enumerate(chunks):
                chunk_hash = hashlib.sha256(chunk.page_content.encode()).hexdigest()
                if self.collection.find_one({"hash": chunk_hash}): continue

                doc = {
                    "text": chunk.page_content,
                    "embedding": self.embedding_model.encode(chunk.page_content).tolist(),
                    "hash": chunk_hash,
                    "metadata": {**(metadata or {}), "chunk_index": i}
                }
                self.collection.insert_one(doc)
                new_chunks += 1
            return new_chunks
        except Exception as e:
            print(f"❌ Ingestion Error: {e}")
            return 0

    # --- GENERATION ---
    def generate_answer(self, query: str):
        self.add_to_memory("user", query)

        # 1. Search Docs
        q_emb = self.embedding_model.encode(query).tolist()
        ctx_sources = []
        try:
            results = list(self.collection.aggregate([
                {"$vectorSearch": {
                    "index": "vector_index",
                    "path": "embedding",
                    "queryVector": q_emb,
                    "numCandidates": 50,
                    "limit": 3
                }},
                {"$project": {"_id": 0, "text": 1, "metadata": 1, "score": {"$meta": "vectorSearchScore"}}}
            ]))
            ctx_sources = results
        except: pass

        # 2. Auto-Web Search
        use_web = False
        web_content = ""
        highest_score = ctx_sources[0]['score'] if ctx_sources else 0

        # If relevance is low (< 0.5) OR no docs found, trigger web search
        if not ctx_sources or highest_score < 0.5:
            use_web = True
            web_content = self.web_search_and_scrape(query)

        # 3. Prompt
        doc_text = "\n\n".join([f"[Doc: {r['metadata'].get('source')}]\n{r['text']}" for r in ctx_sources])

        prompt = f"""
        Answer the user's question.
        - If the document context is relevant, use it.
        - If the document context is missing or irrelevant, use the Web Context.

        Context from Documents:
        {doc_text}

        Context from Web Search:
        {web_content}

        User Question: {query}
        """

        response = self.llm.generate_content(prompt)
        answer = response.text
        self.add_to_memory("assistant", answer)

        return {
            "answer": answer,
            "sources": ctx_sources,
            "web_used": bool(web_content)
        }

# --- INITIALIZATION ---
try: ENV_DB = userdata.get('DB_NAME')
except: ENV_DB = "rag"
try: ENV_COLLECTION = userdata.get('COLLECTION_NAME')
except: ENV_COLLECTION = "rag-collection"
try: ENV_MODEL = userdata.get('GEMINI_MODEL')
except: ENV_MODEL = "gemini-2.0-flash"

rag = AdvancedRAGWithMemoryVisionWeb(
    mongodb_uri=userdata.get('MONGODB_URI'),
    db_name=ENV_DB,
    collection_name=ENV_COLLECTION,
    model_name=ENV_MODEL
)

🔄 Loading Embedding model (BAAI/bge-large-en-v1.5)...
🔄 Connecting to MongoDB (DB: rag
)...
🔄 Initializing gemini-2.5-flash-lite...
✅ System Initialized!


/tmp/ipython-input-3807184204.py:191: ResourceWarning: Unclosed MongoClient opened at:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-packages/tornado/platform/asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.12/asyncio/base_events.py", line 645, in run_forever
    self._run_once()
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1999, in _run_once
    handle._run()
  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *

In [11]:
# @title 4. Initialize & Create Indexes
# Initialize
rag = AdvancedRAGWithMemoryVisionWeb(
    mongodb_uri=MONGODB_URI,
    db_name="rag",
    collection_name="rag-collection"
)

# Create Hash Index for Deduplication (Runs once)
print("🔧 Ensuring hash index exists...")
try:
    rag.collection.create_index("hash", unique=True)
    print("✅ Hash index verified!")
except Exception as e:
    print(f"Note: {e}")

rag.start_new_session()

🔄 Loading Embedding model (BAAI/bge-large-en-v1.5)...
🔄 Connecting to MongoDB Atlas...
🔄 Initializing gemini-2.5-flash-lite...
✅ System Initialized!
🔧 Ensuring hash index exists...
✅ Hash index verified!


'afb2bd9d-987c-42bd-9f6d-9dd30534339b'

In [12]:
# @title 5. Upload & Ingest Documents (FIXED)
print("📤 Upload documents (PDF, DOCX, TXT, CSV)")
uploaded = files.upload()

total_chunks = 0

for filename, content in uploaded.items():
    print(f"Processing {filename}...")

    # Write to temp file
    temp_path = f"/tmp/{filename}"
    with open(temp_path, 'wb') as f:
        f.write(content)

    # Ingest
    chunks_added = rag.ingest_file(temp_path, metadata={"source": filename})

    # Safety Check: Ensure chunks_added is a number before adding
    if chunks_added is not None:
        total_chunks += chunks_added
    else:
        print(f"⚠️ Warning: No chunks returned for {filename}")

    # Cleanup
    if os.path.exists(temp_path):
        os.remove(temp_path)

print(f"\n✅ Ingestion Complete! Added {total_chunks} new chunks.")

📤 Upload documents (PDF, DOCX, TXT, CSV)


Saving Kethan VR A Comprehensive.md to Kethan VR A Comprehensive.md
Processing Kethan VR A Comprehensive.md...

✅ Ingestion Complete! Added 0 new chunks.


In [19]:
# @title 6. Interactive Chat Interface (Zero-Noise Mode)
import sys
import os
import warnings
import logging
from contextlib import contextmanager

# --- NUCLEAR OPTION: SUPPRESS ALL NOISE ---
# 1. Standard Python Warnings
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

# 2. Silence Third-Party Loggers
logging.getLogger("tornado.access").setLevel(logging.CRITICAL)
logging.getLogger("urllib3").setLevel(logging.CRITICAL)
logging.getLogger("duckduckgo_search").setLevel(logging.CRITICAL)

# 3. Context Manager to eat stderr during noisy operations
@contextmanager
def suppress_stderr():
    with open(os.devnull, "w") as devnull:
        old_stderr = sys.stderr
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stderr = old_stderr

from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.table import Table
from rich import box
from google.colab import files

console = Console()

console.print(Panel.fit(
    "[bold cyan]Advanced RAG System[/bold cyan]\n"
    "[dim]Commands:[/dim]\n"
    "• [green]Any question[/green]: Search docs + memory\n"
    "• [green]web: <query>[/green]: Force web search\n"
    "• [green]image[/green]: Upload and analyze image\n"
    "• [green]exit[/green]: Quit",
    title="🚀 Ready (Zero-Noise Mode)", border_style="cyan"
))

while True:
    try:
        # Use standard input() to avoid Colab recursion errors
        print("\n" + "─"*50)
        query = input("User 👤 (Type 'exit' to quit): ").strip()

        if query.lower() in ['exit', 'quit', 'q']:
            console.print("[bold red]👋 Goodbye![/bold red]")
            break

        if not query: continue

        # --- IMAGE MODE ---
        if query.lower() == 'image':
            console.print("[yellow]📤 Upload an image...[/yellow]")
            img_upload = files.upload()
            if img_upload:
                q_img = input("Question about image: ")
                with console.status("[bold green]Analyzing Image...[/bold green]"):
                    with suppress_stderr(): # SILENCE WARNINGS HERE
                        ans = rag.analyze_uploaded_image(img_upload, q_img)
                console.print(Panel(Markdown(ans), title="🖼️ Image Analysis", border_style="magenta"))
            continue

        # --- TEXT/WEB MODE ---
        use_web = False
        if query.startswith("web:"):
            use_web = True
            query = query.replace("web:", "").strip()

        with console.status("[bold green]Thinking...[/bold green]"):
            try:
                # WRAP GENERATION IN SILENCE
                # This eats the "duckduckgo" and "ipywidgets" red text
                with suppress_stderr():
                    if hasattr(rag, 'web_search_and_scrape'):
                         result = rag.generate_answer(query)
                    else:
                         result = rag.generate_answer(query)

                # Display Answer
                console.print(Panel(
                    Markdown(result['answer']),
                    title="🤖 AI Response",
                    border_style="green",
                    box=box.ROUNDED
                ))

                # Display Sources
                if result.get('sources'):
                    table = Table(title="📚 Sources Used", box=box.SIMPLE)
                    table.add_column("Score", style="cyan")
                    table.add_column("Source File", style="magenta")

                    for s in result['sources']:
                        table.add_row(
                            f"{s.get('score', 0):.2f}",
                            s['metadata'].get('source', 'unknown')
                        )
                    console.print(table)

                if result.get('web_used'):
                    console.print("[dim]🌐 Web content was used to answer this.[/dim]")

            except Exception as e:
                # We turn stderr back on for actual errors so you can see them
                console.print(f"[red]Generation Error: {e}[/red]")

    except KeyboardInterrupt:
        console.print("\n[bold red]👋 Goodbye![/bold red]")
        break
    except Exception as e:
        console.print(f"[red]System Error: {e}[/red]")

╭───── 🚀 Ready (Zero-Noise Mode) ─────╮
│ Advanced RAG System                  │
│ Commands:                            │
│ • Any question: Search docs + memory │
│ • web: <query>: Force web search     │
│ • image: Upload and analyze image    │
│ • exit: Quit                         │
╰──────────────────────────────────────╯


──────────────────────────────────────────────────
User 👤 (Type 'exit' to quit): who is modi


Output()

🌐 Searching DuckDuckGo for: 'who is modi'...

╭──────────────────────────────────────────────── 🤖 AI Response ─────────────────────────────────────────────────╮
│ Narendra Modi is the current Prime Minister of India. He assumed office on May 26, 2014, and is a prominent     │
│ leader of the Bharatiya Janata Party (BJP).                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

🌐 Searching DuckDuckGo for: 'kethan?'...

Generation Error: 429 POST 
https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent?%24alt=json%3Benum-en
coding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on 
this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: 
https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, 
model: gemini-2.5-flash-lite
Please retry in 24.506032167s.

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7970fe273a80>



──────────────────────────────────────────────────


👋 Goodbye!